In [15]:
'''
导入包
'''
import tushare as ts
import datetime
from dateutil.relativedelta import relativedelta
import openpyxl
from decimal import Decimal

print('导入包')

导入包


In [16]:
# 全局的业务变量
class BusinessVal:
    # 标的资产，暂时只支持标的资产是一种指数
    underlyingAsset = '000001.SH'
    # 敲入比例
    knockInRatio = Decimal('0.90')
    # 敲出比例
    knockOutRatio = Decimal('1.00')
    # 期限，单位是月
    timeLimit = 3
    # start_date
    startDate = '20190101'
    # end_date
    endDate = '20200531'
    # 月利率
    monthlyRate = Decimal('0.1')
    # 保证金
    deposit = Decimal('20')
    # 名义本金倍数，名义本金除以保证金
    multiple = Decimal('5')

# 全局的技术变量
class TechVal:
    # tushare_token
    tushareToken = '19280d5f247ae49caae39e24a4363308c5396514275535ed770716c2'
    unKnowTradeDate = '无法确定交易日'

print('标的资产' + ' ' * 3 + str(BusinessVal.underlyingAsset))
print('敲出比例' + ' ' * 3 + str(BusinessVal.knockOutRatio))
print('敲入比例' + ' ' * 3 + str(BusinessVal.knockInRatio))
print('期限' + ' ' * 3 + str(BusinessVal.timeLimit))
print('模拟开始日期' + ' ' * 3 + str(BusinessVal.startDate))
print('模拟停止日期' + ' ' * 3 + str(BusinessVal.endDate))
print('月利率' + ' ' * 3 + str(BusinessVal.monthlyRate))
print('保证金' + ' ' * 3 + str(BusinessVal.deposit))
print('杠杆倍数' + ' ' * 3 + str(BusinessVal.multiple))

标的资产   000001.SH
敲出比例   1.00
敲入比例   0.90
期限   3
模拟开始日期   20190101
模拟停止日期   20200531
月利率   0.1
保证金   20
杠杆倍数   5


In [17]:
# 初始化 tushare
ts.set_token(TechVal.tushareToken)
pro = ts.pro_api()

print('初始化tushare')

初始化tushare


In [18]:
'''
获取行情数据
'''
quote = pro.index_daily(ts_code=BusinessVal.underlyingAsset, start_date = BusinessVal.startDate, end_date=BusinessVal.endDate)
# 行反转，这样第一行是最早的日期
quote = quote.reindex(index=quote.index[::-1])
print(quote)

       ts_code trade_date      close       open       high        low  \
340  000001.SH   20190102  2465.2910  2497.8805  2500.2783  2456.4233   
339  000001.SH   20190103  2464.3628  2461.7829  2488.4790  2455.9256   
338  000001.SH   20190104  2514.8682  2446.0193  2515.3160  2440.9066   
337  000001.SH   20190107  2533.0887  2528.6987  2536.9775  2515.5083   
336  000001.SH   20190108  2526.4622  2530.3001  2531.3450  2520.1648   
..         ...        ...        ...        ...        ...        ...   
4    000001.SH   20200525  2817.9697  2816.2410  2821.4978  2802.4653   
3    000001.SH   20200526  2846.5473  2827.8967  2848.3371  2825.9010   
2    000001.SH   20200527  2836.8036  2847.3198  2848.9965  2831.9281   
1    000001.SH   20200528  2846.2217  2838.2072  2861.9241  2820.1455   
0    000001.SH   20200529  2852.3512  2835.5840  2855.3756  2829.6271   

     pre_close   change  pct_chg          vol       amount  
340  2493.8962 -28.6052  -1.1470  109932013.0   97592572.0  
3

In [19]:
'''
定义一个函数，获取最近的一个交易日
'''
# 交易日历
tradeCalendar = pro.trade_cal(is_open='1')
print(tradeCalendar)

lastTradeData = tradeCalendar.iloc[[-1]]['cal_date'].values[0]

def getLeastTradeDate(searchData):
    '''
    获取searchData的在未来最近一个交易日
    :param searchData: 传入日期
    :return: outDate
    '''
    
    if datetime.datetime.strptime(searchData,'%Y%m%d') > datetime.datetime.strptime(lastTradeData,'%Y%m%d'):
        return TechVal.unKnowTradeDate
    
    # 如果结果是空，说明不是交易日
    while (tradeCalendar.loc[tradeCalendar['cal_date'] == searchData]).empty:
        searchData = (datetime.datetime.strptime(searchData,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
        if datetime.datetime.strptime(searchData,'%Y%m%d') > datetime.datetime.strptime(lastTradeData,'%Y%m%d'):
            return TechVal.unKnowTradeDate
    
    rntDate = searchData
    return rntDate

def isTradeDate(checkDate):
    '''
    判断checkData是否是交易日
    :param checkDate: 
    :return: 
    '''
    # 如果结果是空，说明不是交易日
    if (tradeCalendar.loc[tradeCalendar['cal_date'] == checkDate]).empty:
        return False
    else:
        return True
    

     exchange  cal_date  is_open
0         SSE  19901219        1
1         SSE  19901220        1
2         SSE  19901221        1
3         SSE  19901224        1
4         SSE  19901225        1
...       ...       ...      ...
7338      SSE  20201225        1
7339      SSE  20201228        1
7340      SSE  20201229        1
7341      SSE  20201230        1
7342      SSE  20201231        1

[7343 rows x 3 columns]


In [20]:
# 模拟
def simulate(anchorDate,anchorPrice,knockOutPrice,knockInPrice,observeList):
    '''
    模拟
    :param anchorDate: 锚定日期
    :param anchorPrice: 锚定价格
    :param knockOutPrice: 敲出价格
    :param knockInPrice: 敲入价格
    :param observeList: 观察日列表
    :return: 
    '''
    # 是否曾经敲入过
    isKnockIn = False
    # 是否曾经敲出过
    isKnockOut = False
    # 模拟日期
    simulateDate = anchorDate
    # 到期日
    endDate = observeList[-1]
    # 如果没到期
    observeIndex = 0
    # 敲入信息
    knockInList = []
    knockOutList = []
    
    for simulateDate in observeList:
        observeIndex = observeIndex + 1
        # 获取该日期行情数据
        simulateQuote = quote.loc[quote['trade_date'] == simulateDate]
        # 判断是否敲入
        if simulateQuote['close'].values[0] < knockInPrice:
            # 记录敲入信息
            knockInList.append(simulateDate + ':' + str(simulateQuote['close'].values[0]))
            isKnockIn = True
        # 判断是否敲出
        if simulateQuote['close'].values[0] > knockOutPrice:
            knockOutList.append(simulateDate + ':' + str(simulateQuote['close'].values[0]))
            isKnockOut = True
        # 没有敲入，没有敲出
        if isKnockIn == False and (isKnockOut == False and (simulateDate == endDate)):
            payType = '没有敲入，没有敲出'
            payDate = simulateDate
            payOff = BusinessVal.deposit + observeIndex * BusinessVal.deposit * BusinessVal.monthlyRate * BusinessVal.multiple
            # return (payType,payDate,payOff)
            break
        # 没有敲入，有敲出
        if isKnockIn == False and isKnockOut == True:
            payType = '没有敲入，有敲出'
            payDate = simulateDate
            payOff = BusinessVal.deposit + observeIndex * BusinessVal.deposit * BusinessVal.monthlyRate * BusinessVal.multiple
            # return (payType,payDate,payOff)
            break
        # 有敲入，有敲出
        if isKnockIn == True and isKnockOut == True:
            payType = '有敲入，有敲出'
            payDate = simulateDate
            payOff = BusinessVal.deposit + (observeIndex - len(knockInPrice)) * BusinessVal.deposit * BusinessVal.monthlyRate * BusinessVal.multiple
            # return (payType,payDate,payOff)
            break
        # 有敲入，没有敲出
        if isKnockIn == True and (isKnockOut == False and (simulateDate == endDate)):
            payType = '有敲入，没有敲出'
            payDate = simulateDate
            # 跌幅
            dropPercent = (anchorPrice -  Decimal(simulateQuote['close'].values[0]))/anchorPrice
            payOff = BusinessVal.deposit + (observeIndex - len(knockInList)) * BusinessVal.deposit - BusinessVal.deposit * dropPercent * BusinessVal.multiple
            # return (payType,payDate,payOff)
            break
            
    return (payType,payDate,payOff,isKnockIn,isKnockOut,knockInList,knockOutList)


print('定义模拟器')

定义模拟器


In [21]:
wb = openpyxl.Workbook()
wsBase = wb.active
wsBase.title = '基础信息'
wsBase.append(['标的资产',str(BusinessVal.underlyingAsset)])
wsBase.append(['敲入比例',str(BusinessVal.knockInRatio)])
wsBase.append(['敲出比例',str(BusinessVal.knockOutRatio)])
wsBase.append(['期限(月)',str(BusinessVal.timeLimit)])
wsBase.append(['模拟开始日期',str(BusinessVal.startDate)])
wsBase.append(['模拟停止日期',str(BusinessVal.endDate)])
wsBase.append(['月利率',str(BusinessVal.monthlyRate)])
wsBase.append(['保证金',str(BusinessVal.deposit)])
wsBase.append(['杠杆',str(BusinessVal.multiple)])

# 最后一个行情日
lastQuoteDate = quote.iloc[[-1]]['trade_date'].values[0]
wsBase.append(['最后一个行情日',str(lastQuoteDate)])

wsSim = wb.create_sheet(title='模拟结果')
wsSim.append(['锚定日期','锚定价格','敲入价格','敲出价格','敲出观察日列表','结束类型','结束日期','本金收益和','是否有敲入','是否有敲出','敲入次数','敲出次数','敲入信息','敲出信息'])

# 遍历所有行情
for index, row in quote.iterrows():
    # 锚定日期
    anchorDate = row['trade_date']
    print(anchorDate)
    # 锚定价格
    anchorPrice = Decimal(str(row['close']))
    # 敲入价格
    knockInPrice = anchorPrice * BusinessVal.knockInRatio
    # 敲出价格
    knockOutPrice = anchorPrice * BusinessVal.knockOutRatio
    # 观察日列表
    observeList = []
    for i in range(1,BusinessVal.timeLimit + 1,1):
        observeList.append(getLeastTradeDate((datetime.datetime.strptime(anchorDate,'%Y%m%d') + relativedelta(months=i)).strftime('%Y%m%d')))
    appendList = [str(anchorDate), str(anchorPrice),str(knockInPrice),str(knockOutPrice),','.join(observeList)]
    if TechVal.unKnowTradeDate in observeList:
        appendList.append('存在无法确定的观察日，无法进行模拟')
        wsSim.append(appendList)
        continue
    if datetime.datetime.strptime(observeList[-1],'%Y%m%d') > datetime.datetime.strptime(lastQuoteDate,'%Y%m%d'):
        appendList.append('最后一个观察日大于最后一个行情日，无法进行模拟')
        wsSim.append(appendList)
        continue
    payType,payDate,payOff,isKnockIn,isKnockOut,knockInList,knockOutList = simulate(anchorDate,anchorPrice,knockOutPrice,knockInPrice,observeList)
    appendList.append(str(payType))
    appendList.append(str(payDate))
    appendList.append(str(payOff))
    appendList.append((str(isKnockIn)))
    appendList.append((str(isKnockOut)))
    appendList.append((str(len(knockInList))))
    appendList.append((str(len(knockOutList))))
    appendList.append(','.join(knockInList))
    appendList.append(','.join(knockOutList))
    wsSim.append(appendList)
    
fileName = BusinessVal.underlyingAsset + ' ' + '凤凰模拟-' + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.xlsx'
print(fileName)
wb.save(fileName)
quote.to_excel(BusinessVal.underlyingAsset + ' ' + '凤凰行情-' + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.xlsx')

print('FINISH')

20190102
20190103
20190104
20190107
20190108
20190109
20190110
20190111
20190114
20190115
20190116
20190117
20190118
20190121
20190122
20190123
20190124
20190125
20190128
20190129
20190130
20190131
20190201
20190211
20190212
20190213
20190214
20190215
20190218
20190219
20190220
20190221
20190222
20190225
20190226
20190227
20190228
20190301
20190304
20190305
20190306
20190307
20190308
20190311
20190312
20190313
20190314
20190315
20190318
20190319
20190320
20190321
20190322
20190325
20190326
20190327
20190328
20190329
20190401
20190402
20190403
20190404
20190408
20190409
20190410
20190411
20190412
20190415
20190416
20190417
20190418
20190419
20190422
20190423
20190424
20190425
20190426
20190429
20190430
20190506
20190507
20190508
20190509
20190510
20190513
20190514
20190515
20190516
20190517
20190520
20190521
20190522
20190523
20190524
20190527
20190528
20190529
20190530
20190531
20190603
20190604
20190605
20190606
20190610
20190611
20190612
20190613
20190614
20190617
20190618
20190619
2